In [1]:
import torch
import numpy as np

In [2]:
from roarena.corruption import CorruptionJob
from roarena.corruption import CORRUPTIONS

job = CorruptionJob('store', 'datasets', readonly=True)

for animal in ['mouse', 'monkey']:
    if animal=='mouse':
        model_paths = {
            'base': 'store/models/CIFAR10-G_Baseline.pt',
            'neural': 'store/models/CIFAR10-G_Mouse.pt',
        }
    if animal=='monkey':
        model_paths = {
            'base': 'store/models/VGG19_Baseline.pt',
            'neural': 'store/models/VGG19_MTL.pt',
        }
    
    accs = {}
    for label, model_path in model_paths.items():
        saved = torch.load(model_path)
        accs[label] = {0: saved['acc']}
        for severity in range(1, 6):
            accs[label][severity] = []
            for corruption in CORRUPTIONS:
                config = {
                    'model_pth': model_path,
                    'severity': severity,
                    'corruption': corruption,
                }
                key = job.configs.get_key(config)
                accs[label][severity].append(job.previews[key]['acc'])
    torch.save(accs, f'store/figs-data/{animal}_corruption.pt')

In [3]:
from roarena.attack import AttackJob
from jarvis.vision import prepare_datasets

job = AttackJob('store', 'datasets', readonly=True)

metric = 'LI'
targeted = True

for animal in ['mouse', 'monkey']:
    if animal=='mouse':
        model_paths = {
            'base': 'store/models/CIFAR10-G_Baseline.pt',
            'neural': 'store/models/CIFAR10-G_Mouse.pt',
        }
        dataset = prepare_datasets('CIFAR10-Gray', 'datasets')
    if animal=='monkey':
        model_paths = {
            'base': 'store/models/VGG19_Baseline.pt',
            'neural': 'store/models/VGG19_MTL.pt',
        }
        dataset = prepare_datasets('TinyImageNet-Gray', 'datasets')
    loader = torch.utils.data.DataLoader(dataset, batch_size=20)

    imgs, advs = {}, {}
    for label, model_path in model_paths.items():
        print(f'gathering advs for {model_path}...')
        imgs[label] = []
        advs[label] = []
        for b_idx, (_imgs, _) in enumerate(loader):
            if b_idx>=50:
                break

            cond = {
                'model_pth': model_path,
                'batch_idx': b_idx,
                'metric': metric, 'targeted': targeted,
                'name': 'BB',
            }
            _advs, _dists = [], []
            for key, config in job.conditioned(cond):
                result = job.results[key]
                _advs.append(result['advs'])
                _dists.append(result['dists'])
            if len(_dists)==0:
                continue
            imgs[label].append(_imgs)

            _advs = np.array(_advs)
            _dists = np.array(_dists)
            idxs = np.argmin(_dists, axis=0)
            advs[label].append(np.array([_advs[idx, i] for i, idx in enumerate(idxs)]))

            if (b_idx+1)%10==0:
                print('{:2d} batches loaded'.format(b_idx+1))
        imgs[label] = np.concatenate(imgs[label])
        advs[label] = np.concatenate(advs[label])
    torch.save({'imgs': imgs, 'advs': advs}, f'store/figs-data/{animal}_advs.pt')

/foolbox/foolbox/attacks/brendel_bethge.py:624: UserWarning: At the first initialisation the optimizer needs to be compiled. This may take between 20 to 60 seconds.
  warnings.warn(


gathering advs for store/models/CIFAR10-G_Baseline.pt...
10 batches loaded
20 batches loaded
30 batches loaded
40 batches loaded
50 batches loaded
gathering advs for store/models/CIFAR10-G_Mouse.pt...
10 batches loaded
20 batches loaded
30 batches loaded
40 batches loaded
50 batches loaded
gathering advs for store/models/VGG19_Baseline.pt...
10 batches loaded
20 batches loaded
30 batches loaded
40 batches loaded
50 batches loaded
gathering advs for store/models/VGG19_MTL.pt...
10 batches loaded
20 batches loaded
30 batches loaded
40 batches loaded
50 batches loaded


In [4]:
from roarena.einmon import EinMonJob
from scipy.fft import fft2, ifft2

job = EinMonJob('store', 'datasets', readonly=True)

In [5]:
for animal in ['mouse', 'monkey']:
    if animal=='mouse':
        dataset = job.prepare_dataset('CIFAR10-Gray', 0, 0)
        idx = 5796
    if animal=='monkey':
        dataset = job.prepare_dataset('TinyImageNet-Gray', 0, 0)
        idx = 7961
    img_low, label_low = dataset.dataset[dataset.idxs_low[idx]]
    img_high, label_high = dataset.dataset[dataset.idxs_high[idx]]

    alphas = [0, 16, 32, 48, 64, 80]
    imgs_low, imgs_high, imgs_mix = {}, {}, {}
    for i, alpha in enumerate(alphas):
        img_size = img_low.shape[1]
        assert img_low.shape[2]==img_size
        dx, dy = np.meshgrid(np.arange(img_size)/img_size, np.arange(img_size)/img_size)
        dx = np.mod(dx+0.5, 1)-0.5
        dy = np.mod(dy+0.5, 1)-0.5
        mask = ((dx**2+dy**2)**0.5<=alpha/100*0.5).astype(float)

        f_low = fft2(img_low.numpy())
        f_high = fft2(img_high.numpy())
        f_mix = f_low*mask+f_high*(1-mask)
        img_mix = np.real(ifft2(f_mix))
        img_mix = np.clip(img_mix, 0, 1)

        imgs_low[alpha] = np.clip(np.real(ifft2(f_low*mask)), 0, 1)[0]
        imgs_high[alpha] = np.clip(np.real(ifft2(f_high*(1-mask)))+img_high.mean().item(), 0, 1)
        imgs_mix[alpha] = img_mix[0]
    torch.save({
        'imgs_low': imgs_low, 'imgs_high': imgs_high, 'imgs_mix': imgs_mix, 'alphas': alphas,
    }, f'store/figs-data/{animal}_einmon_imgs.pt')
    print(f'EinMon images for {animal} saved.')

EinMon images for mouse saved.
EinMon images for monkey saved.


In [6]:
for animal in ['mouse', 'monkey']:
    if animal=='mouse':
        model_paths = {
            'base': 'store/models/CIFAR10-G_Baseline.pt',
            'neural': 'store/models/CIFAR10-G_Mouse.pt',
        }
    if animal=='monkey':
        model_paths = {
            'base': 'store/models/VGG19_Baseline.pt',
            'neural': 'store/models/VGG19_MTL.pt',
        }
    
    alphas = {}
    accs_low, accs_high = {}, {}
    for label, model_path in model_paths.items():
        print(f'gathering ein-mon results for {model_path}...')
        alphas[label] = set()
        cond = {
            'model_pth': model_path,
        }
        for _, config in job.conditioned(cond):
            alphas[label].add(config['alpha'])
        alphas[label] = sorted(list(alphas[label]))
        accs_low[label], accs_high[label] = [], []
        for alpha in alphas[label]:
            cond['alpha'] = alpha
            _accs_low, _accs_high = [], []
            for key, _ in job.conditioned(cond):
                result = job.results[key]
                _accs_low.append(result['acc_low'])
                _accs_high.append(result['acc_high'])
            accs_low[label].append(_accs_low)
            accs_high[label].append(_accs_high)
    torch.save({
        'alphas': alphas, 'accs_low': accs_low, 'accs_high': accs_high,
    }, f'store/figs-data/{animal}_einmon_accs.pt')

gathering ein-mon results for store/models/CIFAR10-G_Baseline.pt...
gathering ein-mon results for store/models/CIFAR10-G_Mouse.pt...
gathering ein-mon results for store/models/VGG19_Baseline.pt...
gathering ein-mon results for store/models/VGG19_MTL.pt...


In [7]:
from numpy.fft import fft2, fftshift, fftfreq

job = AttackJob('store', 'datasets', readonly=True)

metric = 'LI'
targeted = True

dataset = prepare_datasets('CIFAR10', 'datasets')
loader = torch.utils.data.DataLoader(dataset, batch_size=20)

model_paths = {
    'Baseline': 'store/models/CIFAR10_Baseline.pt',
    'Rebuffi21': 'store/models/CIFAR10_Rebuffi21.pt',
    'Gowal20': 'store/models/CIFAR10_Gowal20.pt',
    'Wu20': 'store/models/CIFAR10_Wu20.pt',
    'Zhang20': 'store/models/CIFAR10_Zhang20.pt',
    'Carmon19': 'store/models/CIFAR10_Carmon19.pt',
    'Sehwag20': 'store/models/CIFAR10_Sehwag20.pt',
    'Cui20': 'store/models/CIFAR10_Cui20.pt',
    'Hendrycks20': 'store/models/CIFAR10_Hendrycks20.pt',
    'Kireev21': 'store/models/CIFAR10_Kireev21.pt',
    'Blur': 'store/models/CIFAR10_Blur.pt',
    'PCA': 'store/models/C10R18-PC512.pt',
}

logps, dists = {}, {}
for label, model_path in model_paths.items():
    print(f'computing log Fourier powers for {model_path}...')
    imgs = []
    advs = []
    for b_idx, (_imgs, _) in enumerate(loader):
        if b_idx>=50:
            break

        cond = {
            'model_pth': model_path,
            'batch_idx': b_idx,
            'metric': metric, 'targeted': targeted,
            'name': 'BB',
        }
        _advs, _dists = [], []
        for key, config in job.conditioned(cond):
            result = job.results[key]
            _advs.append(result['advs'])
            _dists.append(result['dists'])
        if len(_dists)==0:
            continue
        imgs.append(_imgs)

        _advs = np.array(_advs)
        _dists = np.array(_dists)
        idxs = np.argmin(_dists, axis=0)
        advs.append(np.array([_advs[idx, i] for i, idx in enumerate(idxs)]))
    imgs = torch.cat(imgs).numpy()
    advs = np.concatenate(advs)
    
    diffs = imgs-advs
    dists[label] = diffs.max(axis=(1, 2, 3))
    diffs -= diffs.mean(axis=(2, 3), keepdims=True)
    powers = np.abs(fft2(diffs))**2
    powers[..., 0, 0] = np.nan
    logps[label] = fftshift(np.log(powers.mean(axis=(0, 1))))
torch.save({
    'logps': logps, 'dists': dists,
}, f'store/figs-data/CIFAR10_advs.pt')

computing log Fourier powers for store/models/CIFAR10_Baseline.pt...
computing log Fourier powers for store/models/CIFAR10_Rebuffi21.pt...
computing log Fourier powers for store/models/CIFAR10_Gowal20.pt...
computing log Fourier powers for store/models/CIFAR10_Wu20.pt...
computing log Fourier powers for store/models/CIFAR10_Zhang20.pt...
computing log Fourier powers for store/models/CIFAR10_Carmon19.pt...
computing log Fourier powers for store/models/CIFAR10_Sehwag20.pt...
computing log Fourier powers for store/models/CIFAR10_Cui20.pt...
computing log Fourier powers for store/models/CIFAR10_Hendrycks20.pt...
computing log Fourier powers for store/models/CIFAR10_Kireev21.pt...
computing log Fourier powers for store/models/CIFAR10_Blur.pt...
computing log Fourier powers for store/models/C10R18-PC512.pt...


In [8]:
e_job = EinMonJob('store', 'datasets', readonly=True)
c_job = CorruptionJob('store', 'datasets', readonly=True)

alphas = {}
accs_low, accs_high = {}, {}
accs = {}
for label, model_path in model_paths.items():
    print(f'gathering ein-mon results for {model_path}...')
    alphas[label] = set()
    cond = {
        'model_pth': model_path,
    }
    for _, config in e_job.conditioned(cond):
        alphas[label].add(config['alpha'])
    alphas[label] = sorted(list(alphas[label]))
    accs_low[label], accs_high[label] = [], []
    for alpha in alphas[label]:
        cond['alpha'] = alpha
        _accs_low, _accs_high = [], []
        for key, _ in e_job.conditioned(cond):
            result = e_job.results[key]
            _accs_low.append(result['acc_low'])
            _accs_high.append(result['acc_high'])
        accs_low[label].append(_accs_low)
        accs_high[label].append(_accs_high)
        
    accs[label] = []
    for severity in range(1, 6):
        for corruption in CORRUPTIONS:
            config = {
                'model_pth': model_path,
                'severity': severity,
                'corruption': corruption,
            }
            key = c_job.configs.get_key(config)
            accs[label].append(c_job.previews[key]['acc'])
    accs[label] = np.mean(accs[label])
torch.save({
    'alphas': alphas, 'accs_low': accs_low, 'accs_high': accs_high, 'accs': accs,
}, f'store/figs-data/CIFAR10_einmon_accs.pt')

gathering ein-mon results for store/models/CIFAR10_Baseline.pt...
gathering ein-mon results for store/models/CIFAR10_Rebuffi21.pt...
gathering ein-mon results for store/models/CIFAR10_Gowal20.pt...
gathering ein-mon results for store/models/CIFAR10_Wu20.pt...
gathering ein-mon results for store/models/CIFAR10_Zhang20.pt...
gathering ein-mon results for store/models/CIFAR10_Carmon19.pt...
gathering ein-mon results for store/models/CIFAR10_Sehwag20.pt...
gathering ein-mon results for store/models/CIFAR10_Cui20.pt...
gathering ein-mon results for store/models/CIFAR10_Hendrycks20.pt...
gathering ein-mon results for store/models/CIFAR10_Kireev21.pt...
gathering ein-mon results for store/models/CIFAR10_Blur.pt...
gathering ein-mon results for store/models/C10R18-PC512.pt...


In [9]:
alphas = {}
accs_low, accs_high = {}, {}
accs = {}
for label in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30]:
    model_path = 'store/models/C10R18_{:02d}.pt'.format(label)
    print(f'gathering ein-mon results for {model_path}...')
    alphas[label] = set()
    cond = {
        'model_pth': model_path,
    }
    for _, config in e_job.conditioned(cond):
        alphas[label].add(config['alpha'])
    alphas[label] = sorted(list(alphas[label]))
    accs_low[label], accs_high[label] = [], []
    for alpha in alphas[label]:
        cond['alpha'] = alpha
        _accs_low, _accs_high = [], []
        for key, _ in e_job.conditioned(cond):
            result = e_job.results[key]
            _accs_low.append(result['acc_low'])
            _accs_high.append(result['acc_high'])
        accs_low[label].append(_accs_low)
        accs_high[label].append(_accs_high)
        
    accs[label] = []
    for severity in range(1, 6):
        for corruption in CORRUPTIONS:
            config = {
                'model_pth': model_path,
                'severity': severity,
                'corruption': corruption,
            }
            key = c_job.configs.get_key(config)
            accs[label].append(c_job.previews[key]['acc'])
    accs[label] = np.mean(accs[label])
torch.save({
    'alphas': alphas, 'accs_low': accs_low, 'accs_high': accs_high, 'accs': accs,
}, f'store/figs-data/CIFAR10-blur_einmon_accs.pt')

gathering ein-mon results for store/models/C10R18_10.pt...
gathering ein-mon results for store/models/C10R18_11.pt...
gathering ein-mon results for store/models/C10R18_12.pt...
gathering ein-mon results for store/models/C10R18_13.pt...
gathering ein-mon results for store/models/C10R18_14.pt...
gathering ein-mon results for store/models/C10R18_15.pt...
gathering ein-mon results for store/models/C10R18_16.pt...
gathering ein-mon results for store/models/C10R18_17.pt...
gathering ein-mon results for store/models/C10R18_18.pt...
gathering ein-mon results for store/models/C10R18_19.pt...
gathering ein-mon results for store/models/C10R18_20.pt...
gathering ein-mon results for store/models/C10R18_25.pt...
gathering ein-mon results for store/models/C10R18_30.pt...
